## Beginning

In [ ]:
import os
#訓練データの正解データ
from PIL import Image
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline

basepath = "../first_retailing/"

In [ ]:
y = pd.read_csv(basepath+"train_master.tsv", sep="\t")

In [ ]:
#utils
#画像のpixelを作成
def get_img_array(path,resize):
    img_array = Image.open(path).resize([resize,resize]).getdata()
    return img_array

#画像を表示
def show_img(path):
    img_array = Image.open(path).resize([128,128])
    img_array.show()

#精度
def M_accuracy(true, pred):
    M_acc = {}
    for i in range(24):
        y_id = pred[true["category_id"].values==i] == i
        M_acc[i]=np.mean(y_id)
    return  np.mean([j for j in M_acc.values()])


#pickle書き出し(<4GB)
import pickle
def write_pickle(pickle_path,v):
    with open(pickle_path, mode='wb') as f:
        pickle.dump(v, f)
    print("Finished Writing pickle")

#pickle 読み込み 
def read_pickle(pickle_path):
    with open(pickle_path, mode='rb') as f:
        read = pickle.load(f)
    return read

In [ ]:
import numpy as np

resize = 128
X = []
for path in y["file_name"]:
    X.append(np.hstack(get_img_array(basepath + "train/%s"%(path), resize)))
X = pd.DataFrame(X)
X.shape()

In [ ]:
X_prediction_files  = get_jpg_name(basepath+"test/")    #予測するやつ

X_prediction_data = []
for i in range(len(X_prediction_files)):
    X_prediction_data.append(np.hstack(get_img_array(basepath + "test/test_%s.jpg"%(i), resize)))
X_prediction_data = pd.DataFrame(X_prediction_data)

In [ ]:
from sklearn.cross_validation import train_test_split

X_train,X_valid,y_train,y_valid = train_test_split(X, y, test_size=0.5,random_state=0)

## Preprocessing

In [ ]:
def preprocessing_for_keras(X):
    X_n = X/255
    X_n.astype("float32")
    X_n = X_n.values.reshape(X.shape[0], 3, 128, 128)    
    return X_n



## Model

In [ ]:
keras

## Evaluation

In [ ]:
X_train_pred = model.predict(X_train_keras)
X_test_pred  = model.predict(X_test_keras)
train_accuracy = M_accuracy(true=y_train, pred=y_train_pred)
test_accuracy  = M_accuracy(true=y_test, pred=y_test_pred)
print("Train Accuracy: %s\n Test Accuracy: %s"%(train_accuracy, test_accuracy))

## Predict

In [ ]:
def make_submission(path, pred_array,  X_prediction_files):
    #test_filesの順番がsample_submissionとは異なるため、変更する.
    submit_dic = {}
    for i,name in enumerate(X_prediction_files):
        submit_dic[name] = pred_array[i]

    submit_list = [] 
    for i in range(len(submit_dic)):
        submit_list.append(["test_%s.jpg"%(i),submit_dic["test_%s.jpg"%(i)]])    

    pd.DataFrame(submit_list).to_csv(path, index=None,columns=None)
    print("Finished Writing Submission file")

In [ ]:
X_prediction_keras = preprocessing_for_keras(X_prediction_data)
pred_array = model.predict(X_prediction_keras)

make_submission("Desktop/~~.csv", pred_array = pred_array, X_prediction_files = X_prediction_files )